In [1]:
from dataset import Dataset
from classificadores import KNN, DMC, PerceptronSimples

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from typing import Union

In [11]:
# Conjunto de dados Câncer de Mama =================================================
dataset = Dataset.from_file( 
    filepath = r"datasets\wdbc.data", 
    label_column = 1,
    delimiter = ",",  
    column_names = ["ID", "Diagnosis", "radius1", "texture1", "perimeter1", "area1", "smoothness1", "compactness1", "concavity1", "concave_points1", "symmetry1", "fractal_dimension1", "radius2", "texture2", "perimeter2", "area2", "smoothness2", "compactness2", "concavity2", "concave_points2", "symmetry2", "fractal_dimension2", "radius3", "texture3", "perimeter3", "area3", "smoothness3", "compactness3", "concavity3", "concave_points3", "symmetry3", "fractal_dimension3"]
).ensure_numeric_labels().remove_features(["ID"]).normalize()

print(dataset)
dataset._label_index_to_name

Dataset(instâncias=569, features=30, classes=2)


{0: 'M', 1: 'B'}

In [6]:
# Conjunto de dados Coluna Vertebral =================================================
dataset = Dataset.from_file( 
    filepath = r"datasets\column_3C.dat", 
    label_column = -1,
    delimiter = " ",  
    column_names = ["pelvic incidence", "pelvic tilt", "lumbar lordosis angle", "sacral slope", "pelvic radius", "degree spondylolisthesis", "class"]
).ensure_numeric_labels().normalize()

print(dataset)
dataset._label_index_to_name

Dataset(instâncias=310, features=6, classes=3)


{0: 'DH', 1: 'SL', 2: 'NO'}

In [12]:
dataset.data

,Diagnosis,radius1,texture1,perimeter1,area1,smoothness1,compactness1,concavity1,concave_points1,symmetry1,...,radius3,texture3,perimeter3,area3,smoothness3,compactness3,concavity3,concave_points3,symmetry3,fractal_dimension3
0,0.0,0.042075,-0.954684,0.091977,-0.272534,0.187506,0.584075,0.406279,0.462227,0.372727,...,0.241551,-0.716951,0.336620,-0.098604,0.202272,0.238583,0.137220,0.824055,0.196925,-0.162272
1,0.0,0.286289,-0.454853,0.231567,0.003181,-0.420240,-0.636464,-0.592784,-0.302485,-0.240404,...,0.213803,-0.392857,0.079635,-0.129571,-0.304893,-0.690873,-0.614058,0.278351,-0.532821,-0.554244
2,0.0,0.202991,-0.219479,0.191486,-0.101166,0.028618,-0.137967,-0.074977,0.271372,0.019192,...,0.112771,-0.279851,0.016883,-0.250983,-0.032820,-0.229250,-0.280511,0.670103,-0.192588,-0.573134
3,0.0,-0.579819,-0.278323,-0.532997,-0.794189,0.622642,0.622723,0.131209,0.045726,0.552525,...,-0.503380,-0.228145,-0.517307,-0.811984,0.830945,0.628023,0.097284,0.769759,1.000000,0.547422
4,0.0,0.259785,-0.686845,0.261972,-0.021421,-0.139298,-0.304214,-0.072165,0.036779,-0.243434,...,0.039488,-0.752132,0.013895,-0.316850,-0.125272,-0.655170,-0.361022,0.116838,-0.684999,-0.714810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,0.0,0.379999,-0.142374,0.357335,0.132980,0.053895,-0.407889,0.142924,0.380716,-0.327273,...,0.246531,-0.233475,0.152348,-0.094672,-0.077726,-0.642945,-0.343930,0.523024,-0.804849,-0.788666
565,0.0,0.244640,0.253974,0.208071,-0.051962,-0.184436,-0.484571,-0.325211,-0.026740,-0.301010,...,0.121309,0.398188,0.041785,-0.240169,-0.399987,-0.680007,-0.486422,0.118900,-0.602996,-0.851371
566,0.0,-0.089498,0.242475,-0.108424,-0.393765,-0.423671,-0.491320,-0.566495,-0.472962,-0.464646,...,-0.213803,0.178038,-0.240102,-0.538537,-0.435647,-0.452591,-0.456390,-0.025430,-0.742559,-0.696183
567,0.0,0.289129,0.327021,0.331076,-0.048568,0.176672,0.580394,0.646673,0.510934,0.350505,...,0.267165,0.460554,0.336620,-0.195930,0.239252,0.631516,0.499521,0.821306,-0.005717,-0.095369


In [8]:
dataset.determination_matrix()

,pelvic incidence,pelvic tilt,lumbar lordosis angle,sacral slope,pelvic radius,degree spondylolisthesis,class
pelvic incidence,1.000000,0.395875,0.514499,0.664158,0.061248,0.407979,0.000845
pelvic tilt,0.395875,1.000000,0.187282,0.003885,0.001067,0.158277,0.044820
lumbar lordosis angle,0.514499,0.187282,1.000000,0.358069,0.006459,0.284798,0.001351
sacral slope,0.664158,0.003885,0.358069,1.000000,0.117064,0.274127,0.014523
pelvic radius,0.061248,0.001067,0.006459,0.117064,1.000000,0.000680,0.055102
degree spondylolisthesis,0.407979,0.158277,0.284798,0.274127,0.000680,1.000000,0.014286
class,0.000845,0.044820,0.001351,0.014523,0.055102,0.014286,1.000000


In [9]:
dataset.vectorize_labels()
dataset.label_encodings

{np.float64(0.0): array([ 1., -1., -1.]),
 np.float64(1.0): array([-1.,  1., -1.]),
 np.float64(2.0): array([-1., -1.,  1.])}

In [10]:
# Separa o conjunto de dados em treinamento e teste
train_dataset, test_dataset = dataset.split()
train_dataset : Dataset 
test_dataset : Dataset

In [ ]:
q = 6                                   # Número de neurônios ocultos
m = train_dataset.class_count           # Número de neurônios de saída
p = train_dataset.features_count + 1    # Número de entradas da rede
eta = 0.1                               # Taxa de aprendizado

# Função de ativação usada
activation = lambda x: np.tanh(x)
ddx_activation = lambda x: 1 - activation(x) ** 2

# Inicializa o vetor de pesos dos neurônios ocultos
W = np.random.normal( size = (q, p) )

# Inicializa o vetor de pesos dos neurônios de saída
M = np.random.normal( size = (m, q+1) )

# Número de épocas de treinamento
max_epocas = 5000

# Percorre um número qualquer de épocas
for epoca in range( max_epocas ):
    total_erros = 0

    # Para cada época, embaralha o conjunto de treinamento
    shuffled_dataset = train_dataset.shuffle()

    # Percorre os exemplos do conjunto de treinamento
    for index, *features, classe in shuffled_dataset:
        # Vetor que representa a classe 
        real_output = train_dataset.encode_label( classe )

        # Monta o vetor de entrada
        X_bias = np.r_[features, -1]

        # Sentido direto - cálculo da ativação e a saída de cada camada
        U = W @ X_bias          # Ativação de cada neurônio oculto
        Y = activation( U )     # Saída dos neurônios ocultos

        Z = np.r_[ Y, -1 ]      # Prepara as entradas para os neurônios de saída

        A = M @ Z               # Ativação dos neurônio de saída
        O = activation ( A )    # Saída da camada de saída

        # Sentido inverso - atualização dos pesos das camadas
        err = real_output - O 

        # Verifica se houve erros
        if np.any(err != 0):
            # atualiza os pesos da camada de saída
            delta_output = err * ddx_activation( A )                        # (m x 1)
            delta_weights_out = eta * np.outer( delta_output, Z )           # (m x (q+1))
            M = M + delta_weights_out

            # calcula os erros retropagados para a camada oculta oculto
            output_weights_no_bias_T = M[:, :-1].T                          # (q x m)
            backpropagated_error = output_weights_no_bias_T @ delta_output  # (q x 1)

            # atualiza os pesos da camada oculta
            delta_hidden = backpropagated_error * ddx_activation(U)         # (q x 1)
            delta_weights_hidden = eta * np.outer( delta_hidden, X_bias )   # (q x p)
            W = W + delta_weights_hidden

        # Monta um vetor de predição baseado no argmax da saída da rede
        predicted_class = -np.ones_like(O)
        predicted_class[ np.argmax(O) ] = +1

        if np.argmax(O) != np.argmax(real_output):
            total_erros += 1

    if (epoca% round(max_epocas*0.05)) == 0:
        print(f"Época {epoca}: {total_erros} erros. - saída {predicted_class} [{real_output}]")
        

Época 0: 138 erros. - saída [-1.  1. -1.] [[-1.  1. -1.]]
Época 250: 34 erros. - saída [-1.  1. -1.] [[-1.  1. -1.]]


KeyboardInterrupt: 

In [ ]:
PS = PerceptronSimples( train_dataset )
PS.train( 500 )

for index, *point_test, classe in test_dataset:
    classe_prevista = PS.predict( point_test )
    print(f"{index}] Previu {classe_prevista} e era {classe} [{classe_prevista == classe}]")

train_dataset._centroids